In [ ]:
#Importing required libraries
import tweepy
import json
import csv
from dateutil import parser
import os

#Setting the search words, the code will grab tweets that contain below hashtags
WORDS = ['#bitcoin','#XRP','#ripple','#ethereum','#BitcoinCash','#EOS','#tether','#litecoin','#stellar','#TRON','#bitcoin SV','Cardano','binance','#Monero', '#DASH']

#Defining twitter API keys and tokens 
CONSUMER_KEY = os.environ.get('CONSUMER_KEY')
CONSUMER_SECRET = os.environ.get('CONSUMER_SECRET')
ACCESS_TOKEN = os.environ.get('ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = os.environ.get('ACCESS_TOKEN_SECRET')

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

class CustomStreamListener(tweepy.StreamListener):

    def on_data(self, data):

        try:
           # Decode the JSON from Twitter
            datajson = json.loads(data)
            
            #Extract data from the Tweet
            text = datajson['text']
            name = datajson['user']['name']
            screen_name = datajson['user']['screen_name']
            tweet_id = datajson['id']
            created_at = parser.parse(datajson['created_at']) 
            followers = datajson['user']['followers_count']
            friends = datajson['user']['friends_count']
 
            #write tweets to tweets.csv file
            with open('tweets.csv', 'a',encoding="utf-8") as f:
                writer = csv.writer(f)
                writer.writerow([tweet_id,name,screen_name,created_at,text,followers,friends])

        except Exception as e:
           print(e)
        
    def on_error(self, status_code):
        print >> sys.stderr, 'Encountered error with status code:', status_code
        return True # Don't kill the stream

    def on_timeout(self):
        print >> sys.stderr, 'Timeout...'
        return True # Don't kill the stream

# Writing csv titles
with open('tweets.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['tweet_id','name','screen_name','created_at','text','followers','friends'])
    
streamingAPI=tweepy.streaming.Stream(auth, CustomStreamListener(api))
streamingAPI.filter(track=WORDS, languages=["en"])